### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5607,2023-11-06,Brasil Nbb,19:00,Paulistano,Unifacisa,2.25,1.57,148.5,1.83,1.83,2.5,1.83,1.83
5608,2023-11-06,Brasil Nbb,19:30,São José,Pato,1.36,3.05,153.5,1.83,1.83,-6.5,1.83,1.83
5609,2023-11-06,Brasil Nbb,20:00,São Paulo,Fortaleza B.C.,1.18,4.40,156.5,1.83,1.83,-10.5,1.87,1.80
5610,2023-11-06,Eua Nba,21:00,Indiana Pacers,San Antonio Spurs,1.32,3.50,237.5,1.91,1.91,-7.5,1.91,1.91
5611,2023-11-06,Eua Nba,21:30,Brooklyn Nets,Milwaukee Bucks,2.80,1.45,230.5,1.91,1.91,5.5,1.91,1.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5724,2023-11-06,Turquia Superliga,13:00,Merkezefendi,Petkim Spor,2.26,1.62,165.5,1.81,1.92,1.5,2.06,1.68
5725,2023-11-06,Turquia Tbl,12:00,OGM Ormanspor,Konyaspor,1.80,1.90,167.5,1.83,1.83,-2.5,2.09,1.66
5726,2023-11-06,Turquia Tb2L,12:00,Istanbul Anka Spor,Sertas Spor,1.07,7.09,158.5,1.83,1.83,-19.5,2.06,1.35
5727,2023-11-06,Turquia Tb2L,12:00,Tesvikiye,Istanbul Basket,2.45,1.50,155.5,1.83,1.83,2.5,2.09,1.66


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,21:00,Eua Nba,Indiana Pacers,San Antonio Spurs,237.5,1.91,0.7816,Over
4,21:30,Eua Nba,Brooklyn Nets,Milwaukee Bucks,230.5,1.91,0.7857,Over
5,21:30,Eua Nba,Miami Heat,Los Angeles Lakers,222.5,1.91,0.7991,Over
6,16:00,Alemanha Bbl,Tubingen,Alba Berlin,175.5,1.91,0.8575,Over
9,07:00,Coreia Do Sul Kbl,Suwon KT,Goyang Sono,163.5,1.91,0.6864,Over
10,07:00,Coreia Do Sul Wkbl Feminina,Samsung Blue Minx F,Hana 1Q F,137.5,1.91,0.8470,Over
11,18:30,Eua Ncaa,Oregon,Georgia,140.5,1.91,0.8480,Over
12,20:30,Eua Ncaa,Purdue,Samford,143.5,1.91,0.8993,Over
13,21:00,Eua Ncaa,Coll Of Charltn,Iona,150.5,1.91,0.8440,Over
14,21:00,Eua Ncaa,Dayton,Siu Edwardsville,135.5,1.91,0.8730,Over


### AH Zebra V1 (LR)

In [9]:
# Duplicando o dataset
df_ah_pos_v1 = jogos.copy()

# AH
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v1['Zebra'] = df_ah_pos_v1['Zebra'].astype('category')
# Odds
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v1['HA_Odds_H']
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v1['HA_Odds_A']

df_ah_pos_v1.reset_index(drop=True, inplace=True)
df_ah_pos_v1.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v1.empty:
    mdl_ah_pos_v1 = load_model('ML/ah_pos/v1/v1_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v1, df_ah_pos_v1)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] > 1.85))]
    prev = prev[((prev['Over_Line'] >= 200) & (prev['Over_Line'] < 212)) | ((prev['Over_Line'] >= 215) & (prev['Over_Line'] < 218))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V1 (LR) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V1 (LR) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
50,22:00,Eua Nba,Houston Rockets,Sacramento Kings,-2.5,1.88,0.5091,AH Zebra


### AH Zebra V2 (ADA)

In [10]:
# Duplicando o dataset
df_ah_pos_v2 = jogos.copy()

# AH
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v2['Zebra'] = df_ah_pos_v2['Zebra'].astype('category')
# Odds
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v2['HA_Odds_H']
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v2['HA_Odds_A']

df_ah_pos_v2.reset_index(drop=True, inplace=True)
df_ah_pos_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v2.empty:
    mdl_ah_pos_v2 = load_model('ML/ah_pos/v2/v2_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v2, df_ah_pos_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V2 (ADA) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V2 (ADA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
3,21:00,Eua Nba,Indiana Pacers,San Antonio Spurs,-7.5,1.91,0.5049,AH Zebra
15,21:00,Eua Ncaa,Duquesne,Cleveland State,-9.5,1.91,0.6317,AH Zebra
16,21:00,Eua Ncaa,George Mason,Monmouth (N.J.),-11.5,1.91,0.6317,AH Zebra
17,21:00,Eua Ncaa,Notre Dame,Niagara,-8.5,1.91,0.6315,AH Zebra
20,21:00,Eua Ncaa,Rutgers,Princeton,-7.5,1.91,0.6339,AH Zebra
23,21:00,Eua Ncaa,West Virginia,Missouri State,-7.5,1.91,0.6337,AH Zebra
25,21:30,Eua Ncaa,Belmont,Georgia State,-7.5,1.91,0.6337,AH Zebra
27,21:30,Eua Ncaa,Georgia Tech,Georgia Southern,-10.5,1.91,0.6317,AH Zebra
43,23:00,Eua Ncaa,California,St. Thomas (Minn.),-9.5,1.91,0.6315,AH Zebra
75,21:00,Eua Ncaa,Rhode Island,Central Conn. St.,-11.5,1.91,0.6322,AH Zebra


### AH Zebra V3 (ADA - SEED 8941)

In [11]:
# Duplicando o dataset
df_ah_pos_v3 = jogos.copy()

# AH
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v3['Zebra'] = df_ah_pos_v3['Zebra'].astype('category')
# Odds
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v3['HA_Odds_H']
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v3['HA_Odds_A']

df_ah_pos_v3.reset_index(drop=True, inplace=True)
df_ah_pos_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v3.empty:
    mdl_ah_pos_v3 = load_model('ML/ah_pos/v3/v3_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v3, df_ah_pos_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V3 (ADA - SEED 8941) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V3 (ADA - SEED 8941) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
3,21:00,Eua Nba,Indiana Pacers,San Antonio Spurs,-7.5,1.91,0.5044,AH Zebra


### AH Zebra V4 (ADA - SEED 4280)

In [12]:
# Duplicando o dataset
df_ah_pos_v4 = jogos.copy()

# AH
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v4['Zebra'] = df_ah_pos_v4['Zebra'].astype('category')
# Odds
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v4['HA_Odds_H']
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v4['HA_Odds_A']

df_ah_pos_v4.reset_index(drop=True, inplace=True)
df_ah_pos_v4.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v4.empty:
    mdl_ah_pos_v4 = load_model('ML/ah_pos/v3/v4_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v4, df_ah_pos_v4)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v4_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
3,21:00,Eua Nba,Indiana Pacers,San Antonio Spurs,-7.5,1.91,0.5032,AH Zebra
15,21:00,Eua Ncaa,Duquesne,Cleveland State,-9.5,1.91,0.5850,AH Zebra
16,21:00,Eua Ncaa,George Mason,Monmouth (N.J.),-11.5,1.91,0.5850,AH Zebra
17,21:00,Eua Ncaa,Notre Dame,Niagara,-8.5,1.91,0.5850,AH Zebra
20,21:00,Eua Ncaa,Rutgers,Princeton,-7.5,1.91,0.5883,AH Zebra
23,21:00,Eua Ncaa,West Virginia,Missouri State,-7.5,1.91,0.5883,AH Zebra
25,21:30,Eua Ncaa,Belmont,Georgia State,-7.5,1.91,0.5859,AH Zebra
27,21:30,Eua Ncaa,Georgia Tech,Georgia Southern,-10.5,1.91,0.5850,AH Zebra
43,23:00,Eua Ncaa,California,St. Thomas (Minn.),-9.5,1.91,0.5850,AH Zebra
75,21:00,Eua Ncaa,Rhode Island,Central Conn. St.,-11.5,1.91,0.5850,AH Zebra
